In [33]:
'''
This is code for extracting NN features of face image data 
and then fit a linear model to predict attractiveness of a face
Available dataset: TWIN, CHICAGO and MIT
Available NN feature: 'caffeNet','vgg16','vggFace' and 'faceSNN'

BY Linjie Li
Please run this code on guru2 server
'''
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import os
import random
from os import listdir
from os.path import isfile, join
from sklearn.tree import DecisionTreeRegressor
# Load image dataset#
Dataset = 'mit' # 'twin', 'chicago' or 'mit'
ratingPath = '../Result/'+Dataset+'/meanRating.csv'
socialFea = '../Result/'+Dataset+'/socialMeasures.csv'
mean_rating = pd.read_csv(ratingPath,index_col = 0).as_matrix()[:,0].tolist()
mean_rating = map(float, mean_rating)
mean_rating = np.array(mean_rating)



feature_new = pd.read_csv(socialFea,index_col = 0)
#print feature_new.columns
feature_new = feature_new.drop([ 'catch', 'catchAns', 'subage', 'submale', 'subrace',\
'catch.1', 'catchAns.1', 'subage.1', 'submale.1', 'subrace.1','unattractive','attractive'],axis = 1)
print feature_new.columns
feature_new = feature_new.as_matrix()
print feature_new

Index([u'atypical', u'boring', u'calm', u'cold', u'common', u'confident',
       u'egotistic', u'emotUnstable', u'forgettable', u'intelligent',
       u'introverted', u'kind', u'responsible', u'trustworthy', u'unemotional',
       u'unfamiliar', u'unfriendly', u'unhappy', u'weird', u'aggressive',
       u'caring', u'emotStable', u'emotional', u'familiar', u'friendly',
       u'happy', u'humble', u'interesting', u'irresponsible', u'mean',
       u'memorable', u'normal', u'sociable', u'typical', u'uncertain',
       u'uncommon', u'unintelligent', u'untrustworthy'],
      dtype='object')
[[ 4.285714  4.2       5.933333 ...,  3.666667  2.933333  2.266667]
 [ 3.333333  4.733333  5.866667 ...,  4.333333  3.2       3.666667]
 [ 2.666667  3.866667  5.933333 ...,  3.6       3.8       3.666667]
 ..., 
 [ 4.066667  5.        6.066667 ...,  3.        2.666667  3.866667]
 [ 4.4       5.066667  5.066667 ...,  4.066667  3.066667  5.4     ]
 [ 3.8       3.533333  5.333333 ...,  4.533333  3.733333  3.8

In [35]:
corrList = []
varList = []
mseList = []
from sklearn.cross_validation import KFold
imgNum = feature_new.shape[0]
foldNum = 20
kf = KFold(imgNum, n_folds=foldNum, shuffle=True)
print(kf)
for train_index, test_index in kf:
    feature_train, feature_test = feature_new[train_index], feature_new[test_index]
    rating_train, rating_test = mean_rating[train_index], mean_rating[test_index]
    
    # Do linear regression on feature_arr and mean_rating
    regr = DecisionTreeRegressor(random_state=0)
    regr.fit(feature_train, rating_train)
    predicted_rating = regr.predict(feature_test)

    # Calculate the mean square error
    MSE = np.mean((predicted_rating - rating_test) ** 2)
    mseList.append(MSE)
    
    # Returns the coefficient of determination R^2 of the prediction.
    variance_score = regr.score(feature_test, rating_test)
    varList.append(variance_score)
    
    # Calculate the correlation between prediction and actual rating.
    cor = np.corrcoef(predicted_rating, rating_test)
    corrList.append(cor[0,1])
# print mseList
# print varList
# print corrList
print 'Residual sum of squares: %.2f' % (sum(mseList)/foldNum)
print 'Variance score is: %.2f' % (sum(varList)/foldNum)
print 'Correlation between predicted ratings and actual ratings is: %.4f'%(sum(corrList)/foldNum)


sklearn.cross_validation.KFold(n=2222, n_folds=20, shuffle=True, random_state=None)
Residual sum of squares: 1.07
Variance score is: 0.25
Correlation between predicted ratings and actual ratings is: 0.6226
